# TM10007 Assignment

In [4]:
# Run this to use from colab environment
!pip install -q --upgrade git+https://github.com/jveenland/tm10007_ml.git

  Preparing metadata (setup.py) ... done
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 363.4/363.4 MB 4.2 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 13.8/13.8 MB 60.4 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 24.6/24.6 MB 46.7 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 883.7/883.7 kB 36.5 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 664.8/664.8 MB 2.4 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 211.5/211.5 MB 4.6 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 56.3/56.3 MB 11.6 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 127.9/127.9 MB 6.9 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 207.5/207.5 MB 5.6 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 21.1/21.1 MB 64.7 MB/s eta 0:00:00


## Import packages

In [5]:
import pandas as pd

from worcgist.load_data import load_data

## 1. Data loading

Below are functions to load the dataset of your choice. After that, it is all up to you to create and evaluate a classification method. Beware, there may be missing values in these datasets. Good luck!

In [6]:
# Load data
data = load_data()

data['label'] = data['label'].map({'non-GIST': 0, 'GIST': 1})

y = data['label']
x = data.drop(columns=['label'])

# Calculate basic statistics
n_samples = len(y)
n_features = len(x.columns)

n_missing = data.isna().sum().sum()
n_zero = (data == 0).sum().sum()

n_non_gist = (y == 0).sum().sum()
n_gist = (y == 1).sum().sum()
# Print basic statistics
print(f'The number of samples: {n_samples}')
print(f'The number of features: {n_features}')
print(f'The number of datapoints: {n_samples * n_features}')
print(f"The number of missing values: {n_missing}")
print(f"The number of zero values: {n_zero}")
print(f"The percentage of zero values: {n_non_gist/(n_samples*n_features)*100} %")
print(f"The percentage of GIST tumors: {n_gist/n_samples*100} %")
print(f"The percentage of non-GIST tumors: {n_non_gist/n_samples*100} %")
#data.info()
#data.describe()
#data.duplicated()

The number of samples: 246
The number of features: 493
The number of datapoints: 121278
The number of missing values: 0
The number of zero values: 6848
The percentage of zero values: 0.0997707745840136 %
The percentage of GIST tumors: 50.81300813008131 %
The percentage of non-GIST tumors: 49.1869918699187 %


## 2. Pre-processing

### Remove constants

In [7]:
# Remove columns with constant values
constant_cols = x.columns[x.nunique() <= 1] # get columns with 1 unique value or all values NaN
x = x.drop(columns=constant_cols) # drop these columns

# Remove rows with constant values
constant_rows = x.index[x.nunique(axis=1) <= 1] # get rows with 1 unique value or all values NaN
x_clean = x.drop(index=constant_rows) # drop these columns

# Adjust y to match the remainng samples
y_clean = y.drop(index=constant_rows)

# Combine x_clean and y_clean to one DataFrame
data_constants_removed = pd.concat([x_clean, y_clean], axis=1)

# Get dimensions
dimensions_data_constants_removed = data_constants_removed.shape
n_samples_constants_removed, n_features_constants_removed = dimensions_data_constants_removed

# Change in dimension
samples_removed = n_samples - n_samples_constants_removed
features_removed = n_features - n_features_constants_removed

# Result
print(f'Number of removed samples: {samples_removed}')
print(f'Number of removed features: {features_removed}')
print(f'Dataset without constant values: {data_constants_removed.shape}')

data_constants_removed.head()

Number of removed samples: 0
Number of removed features: 12
Dataset without constant values: (246, 481)


,PREDICT_original_sf_compactness_avg_2.5D,PREDICT_original_sf_compactness_std_2.5D,PREDICT_original_sf_rad_dist_avg_2.5D,PREDICT_original_sf_rad_dist_std_2.5D,PREDICT_original_sf_roughness_avg_2.5D,PREDICT_original_sf_roughness_std_2.5D,PREDICT_original_sf_convexity_avg_2.5D,PREDICT_original_sf_convexity_std_2.5D,PREDICT_original_sf_cvar_avg_2.5D,PREDICT_original_sf_cvar_std_2.5D,...,PREDICT_original_phasef_phasesym_mean_WL3_N5,PREDICT_original_phasef_phasesym_median_WL3_N5,PREDICT_original_phasef_phasesym_std_WL3_N5,PREDICT_original_phasef_phasesym_skewness_WL3_N5,PREDICT_original_phasef_phasesym_kurtosis_WL3_N5,PREDICT_original_phasef_phasesym_range_WL3_N5,PREDICT_original_phasef_phasesym_energy_WL3_N5,PREDICT_original_phasef_phasesym_quartile_range_WL3_N5,PREDICT_original_phasef_phasesym_entropy_WL3_N5,label
ID,,,,,,,,,,,,,,,,,,,,,
GIST-001_0,0.797427,0.120594,80.192445,9.445741,7.102210,3.876595,0.927623,0.081385,0.015227,0.010465,...,0.005697,0.0,0.032461,7.353902,63.183181,0.123189,1591.133826,0.0,15.687095,1
GIST-002_0,0.896991,0.045854,35.323607,2.781343,5.339906,1.923100,0.975762,0.017550,0.009118,0.009385,...,0.009552,0.0,0.043431,6.452863,52.634005,0.172040,220.543010,0.0,12.664845,0
GIST-003_0,0.893238,0.051894,17.582541,1.864908,7.622507,1.039745,0.979608,0.017926,0.013258,0.010217,...,0.022104,0.0,0.068175,3.849719,16.277217,0.286710,62.150817,0.0,10.432943,1
GIST-004_0,0.914027,0.032078,21.951056,2.188093,4.757274,1.786309,0.981125,0.013734,0.011338,0.005147,...,0.010130,0.0,0.044148,5.725807,38.600415,0.182812,39.375540,0.0,10.194878,0
GIST-005_0,0.659747,0.266226,20.801224,4.025831,14.404912,7.695865,0.865275,0.171213,0.048078,0.047425,...,0.012287,0.0,0.043369,4.546357,23.831176,0.183349,24.192700,0.0,10.066832,0


In [8]:
from scipy.stats import shapiro

def check_normal_distribution(df, alpha=0.05):
    """
    Check if each column in the given DataFrame follows a normal distribution.

    Parameters:
    df (pd.DataFrame): The input DataFrame containing numerical data.
    alpha (float): The significance level for the Shapiro-Wilk test.

    Returns:
    pd.DataFrame: A DataFrame indicating whether each column in the input DataFrame follows a normal distribution.
    The DataFrame has one row and the same number of columns as the input DataFrame.
    """
    results = {}
    for col in df.columns:
        p = shapiro(df[col])[1]
        results[col] = p > alpha
    return pd.DataFrame([results])

results = check_normal_distribution(x)
#print(results)

In [9]:
def replace_outliers(df, is_normal):
    """
    Replace outliers in the given DataFrame based on the distribution type of each column.

    Parameters:
    df (pd.DataFrame): The input DataFrame containing numerical data.
    is_normal (pd.Series): A Series indicating whether each column in the DataFrame is normally distributed (True) or not (False).

    Returns:
    pd.DataFrame: The DataFrame with outliers replaced by the respective bounds.
    list: A list of all outlier values found.
    """
    threshold_nd = 1  # z-score threshold for normally distributed data
    threshold_nnd = 5  # IQR threshold for non-normally distributed data
    outliers_list = []  # Initialize an empty list to store outliers

    for column in df.columns:
        if is_normal[column]:
            # Use z-score for normally distributed columns
            mean = df[column].mean()
            std = df[column].std()
            lower_bound = mean - threshold_nd * std
            upper_bound = mean + threshold_nd * std
            outliers = (df[column] > upper_bound) | (df[column] < lower_bound)
            outliers_list.extend(df.loc[outliers, column].tolist())  # Append outliers to the list
            df.loc[df[column] > upper_bound, column] = upper_bound
            df.loc[df[column] < lower_bound, column] = lower_bound
        else:
            # Use IQR for non-normally distributed columns
            Q1 = df[column].quantile(0.25)
            Q3 = df[column].quantile(0.75)
            IQR = Q3 - Q1
            lower_bound = Q1 - threshold_nnd * IQR
            upper_bound = Q3 + threshold_nnd * IQR
            outliers = (df[column] > upper_bound) | (df[column] < lower_bound)
            outliers_list.extend(df.loc[outliers, column].tolist())  # Append outliers to the list
            df.loc[df[column] > upper_bound, column] = upper_bound
            df.loc[df[column] < lower_bound, column] = lower_bound

    return df, outliers_list  # Return the modified DataFrame and the outliers list
df_no_outliers, outliers = replace_outliers(x, results.iloc[0])
print("Max of original data (first 5 columns):")
print(x.iloc[:, 10:20].describe().loc['max'])
print("Max of data without outliers (first 5 columns):")
print(df_no_outliers.iloc[:, 10:20].describe().loc['max'])
print("Number of outliers:", len(outliers))

<ipython-input-9-56050a730009>:37: FutureWarning: Setting an item of incompatible dtype is deprecated and will raise an error in a future version of pandas. Value '87.25' has dtype incompatible with int64, please explicitly cast to a compatible dtype first.
  df.loc[df[column] > upper_bound, column] = upper_bound
<ipython-input-9-56050a730009>:37: FutureWarning: Setting an item of incompatible dtype is deprecated and will raise an error in a future version of pandas. Value '75.5' has dtype incompatible with int64, please explicitly cast to a compatible dtype first.
  df.loc[df[column] > upper_bound, column] = upper_bound
<ipython-input-9-56050a730009>:26: FutureWarning: Setting an item of incompatible dtype is deprecated and will raise an error in a future version of pandas. Value '28.18683538209948' has dtype incompatible with int64, please explicitly cast to a compatible dtype first.
  df.loc[df[column] > upper_bound, column] = upper_bound
<ipython-input-9-56050a730009>:26: FutureWar

Max of original data (first 5 columns):
PREDICT_original_sf_prax_avg_2.5D            0.816314
PREDICT_original_sf_prax_std_2.5D            0.169415
PREDICT_original_sf_evar_avg_2.5D            0.092291
PREDICT_original_sf_evar_std_2.5D            0.056980
PREDICT_original_sf_solidity_avg_2.5D        1.247305
PREDICT_original_sf_solidity_std_2.5D        0.230570
PREDICT_original_sf_area_avg_2.5D        21935.647267
PREDICT_original_sf_area_max_2.5D        36947.705399
PREDICT_original_sf_area_min_2.5D         1895.417736
PREDICT_original_sf_area_std_2.5D        10747.729719
Name: max, dtype: float64
Max of data without outliers (first 5 columns):
PREDICT_original_sf_prax_avg_2.5D            0.816314
PREDICT_original_sf_prax_std_2.5D            0.169415
PREDICT_original_sf_evar_avg_2.5D            0.092291
PREDICT_original_sf_evar_std_2.5D            0.056980
PREDICT_original_sf_solidity_avg_2.5D        1.247305
PREDICT_original_sf_solidity_std_2.5D        0.230570
PREDICT_original_sf_ar

## 3. Feature selection

In [11]:
from sklearn.feature_selection import SelectKBest, f_classif

selector = SelectKBest(score_func=f_classif, k=50)
x_reduced_array = selector.fit_transform(x, y)
selected_feature_names = x.columns[selector.get_support()]
x_reduced = pd.DataFrame(x_reduced_array, columns=selected_feature_names)

/usr/local/lib/python3.11/dist-packages/sklearn/feature_selection/_univariate_selection.py:111: UserWarning: Features [ 76  82  86  88  92  93  97 104 106 315 328 408 415 416 421 428 429 434
 441 442 462 472 478] are constant.
  warnings.warn("Features %s are constant." % constant_features_idx, UserWarning)
/usr/local/lib/python3.11/dist-packages/sklearn/feature_selection/_univariate_selection.py:112: RuntimeWarning: invalid value encountered in divide
  f = msb / msw


In [12]:
# prompt: Use principle component analysis to reduce the features of dataset x

from sklearn.decomposition import PCA

# Initialize PCA with the desired number of components
pca = PCA(n_components=10)  # Example: Reduce to 10 principal components

# Fit and transform the data
x_pca = pca.fit_transform(x_reduced)

# Create a new DataFrame with the reduced features
x_pca_df = pd.DataFrame(data=x_pca)
print(x_pca_df.head())


              0             1             2             3             4  \
0  2.706137e+10  1.212581e+09 -1.071573e+08 -8.559836e+07  1.307341e+08   
1 -7.052720e+09 -1.584604e+08  3.666911e+07 -1.276554e+07  8.715630e+07   
2 -7.432557e+09 -2.930830e+08  9.171578e+06 -1.138701e+07 -2.323273e+07   
3 -7.317085e+09 -6.957856e+07 -1.317338e+08 -3.193084e+08 -2.238159e+08   
4 -7.465845e+09 -2.920298e+08  9.899171e+06 -8.641958e+06 -2.155115e+07   

              5             6             7            8           9  
0 -1.527502e+07  9.998871e+07  8.178967e+05  3573.884319 -195.247268  
1 -2.313805e+07  3.365002e+07 -1.431805e+07  1235.259812  354.766085  
2 -3.419513e+06  4.208261e+04 -3.518983e+06 -1647.636857   -5.065317  
3 -9.569021e+07  3.281032e+07 -1.403831e+06 -2749.437694  -16.401380  
4 -1.765979e+06 -1.099826e+06 -3.644305e+06 -1224.667910   89.109313  


## 4. Model training

### 4.1. Data splitting

In [ ]:
from sklearn.model_selection import train_test_split

# Split data into train partition and test partition
x_train, x_test, y_train, y_test = train_test_split(x, y, random_state=0, test_size=0.7)

## 5. Model evaluation